In [1]:
from fastai.data.all import *
from fastai.vision.all import *
import matplotlib.pyplot as plt
import common

In [16]:
path = Path("./target")
fnames = get_image_files(path)

def get_y(fname):
    fname = fname.with_suffix('.csv')
    return [c for xs in common.get_vertices(fname) for x in xs for c in x]

blocks = (ImageBlock(PILImageBW), RegressionBlock(2))
splitter = TrainTestSplitter(test_size=0.25)
dblock = DataBlock(blocks=blocks, get_items=get_image_files, get_y=get_y, splitter=splitter)

dsets = dblock.datasets(path)

dls = dblock.dataloaders(path)

In [17]:
images, labels = first(dls.train)
print(images.shape, labels.shape)

torch.Size([64, 1, 256, 256]) torch.Size([64, 2])


In [18]:
xb,yb = first(dls.valid, bs=64)
xb.shape

torch.Size([64, 1, 256, 256])

In [22]:
xb,yb = to_cpu(xb),to_cpu(yb)

In [45]:
def conv(ni, nf, ks=7, act=True):
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res

simple_cnn = sequential(
    conv(1 ,4),            #14x14
    conv(4 ,8),            #7x7
    conv(8 ,16),           #4x4
    conv(16,32),           #2x2
    conv(32,2, act=False) #1x1
)

simple_cnn = add_head(simple_cnn, 2, 2)

In [46]:
simple_cnn(xb).shape

torch.Size([64, 2])

In [47]:
learn = Learner(dls, simple_cnn, loss_func=F.mse_loss, metrics=accuracy)

In [48]:
learn.summary()

Sequential (Input shape: 64 x 1 x 256 x 256)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 4 x 128 x 128  
Conv2d                                    200        True      
ReLU                                                           
____________________________________________________________________________
                     64 x 8 x 64 x 64    
Conv2d                                    1576       True      
ReLU                                                           
____________________________________________________________________________
                     64 x 16 x 32 x 32   
Conv2d                                    6288       True      
ReLU                                                           
____________________________________________________________________________
                     64 x 32 x 16 x 16   
Conv2d                                    25120      True      
ReLU                                        

In [49]:
learn.fit_one_cycle(2, 0.01)

epoch,train_loss,valid_loss,accuracy,time


AssertionError: Exception occured in `Recorder` when calling event `after_batch`:
	==:
64
128